# Исследование надежности заемщиков


**Цель исследования**: предобрабоать данные, провести анализ.

**Результата исследования**: необходимо ответить на следующие вопросы:

                            1.Есть ли зависимость между количеством детей и возвратом кредита в срок?
                            2.Есть ли зависимость между семейным положением и возвратом кредита в срок?
                            3.Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
                            4.Как разные цели кредита влияют на его возврат в срок?

## Изучение общей информации о данных

**1. Загрузка данных. Путь к файлу:**

`/datasets/data.csv`

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**2. Ознакомление с таблицей.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**3. Основная информация о датафрейме .**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**4. Количество пропусков для каждого столбца.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**5. Заполним пропуски в столбце `total_income` медианным значением, опираясь на тип занятости (столбец `income_type`).**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**6. Заменим отрицательные значения в столбце `days_employed` (трудовой стаж) на положительные.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**7. Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

**8. Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**9. Удалим аномалии в данном столбце.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**10. Проверка удаления аномалий.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**11. Заполним пропуски в столбце `days_employed` медианными значениями в зависимости от типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**12. Проверим, все ли пропуски заполнены**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**13. Заменим вещественный тип данных в столбце `total_income` на целочисленный.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**14. Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв.**

In [14]:
data['education'] = data['education'].str.lower()

**15. Обратоям явные дубликаты.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**16. На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.




In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**17. Выведим перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**18. Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.



In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)


### Шаг 3. Исследование данных и ответы на вопросы исследования

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [23]:
def children_influence (x):
    result = {
    'clients_quantity': x['children'].count(),
    'percent_of_return': (1 - x['debt'].mean()) * 100
    }
    return pd.Series(result).round(2)
data.groupby('children').apply(children_influence)

,clients_quantity,percent_of_return
children,,
0,14091.0,92.46
1,4808.0,90.77
2,2052.0,90.55
3,330.0,91.82
4,41.0,90.24
5,9.0,100.00


In [24]:
five_children = data[data['children'] == 5].count()
all_clients = data['children'].count()
print(five_children/all_clients*100)

children                 0.042192
days_employed            0.042192
dob_years                0.042192
education                0.042192
education_id             0.042192
family_status            0.042192
family_status_id         0.042192
gender                   0.042192
income_type              0.042192
debt                     0.042192
total_income             0.042192
purpose                  0.042192
total_income_category    0.042192
purpose_category         0.042192
dtype: float64


### Влияние количества детей на возврат кредита в срок:
**Вывод:** Клиенты без детей не задерживают возврат кредита чаще остальных (92,46%). Несмотря на то, что клиенты с 5 детьми не задерживали платежи никогда (возврат 100%), объективно оценить данный показатель невозможно, т.к. количество клиентов данной категории составляет 0,04 % от общего количества клиентов.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
def fam_status_influence (x):
    result = {
    'clients_quantity': x['family_status'].count(),
    'percent_of_return': (1 - x['debt'].mean()) * 100
    }
    return pd.Series(result).round(2)
data.groupby('family_status').apply(fam_status_influence)

,clients_quantity,percent_of_return
family_status,,
Не женат / не замужем,2796.0,90.24
в разводе,1189.0,92.94
вдовец / вдова,951.0,93.38
гражданский брак,4134.0,90.69
женат / замужем,12261.0,92.44


### Влияние семейного положения на возврат кредита в срок
**Вывод:** Самые низкие проценты по возвратам в срок у неженатых людей (90,24%) и людей, проживающих в гражданском браке (90,69%). У остальных категорий процент возврата по кредитам составляет 92-93%. Можно сделать вывод, что узаконивание отношений увеличивает вероятность возврата кредита в срок.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
def income_influence (x):
    result = {
    'clients_quantity': x['total_income_category'].count(),
    'percent_of_return': (1 - x['debt'].mean()) * 100
    }
    return pd.Series(result).round(2)
data.groupby('total_income_category').apply(income_influence)

,clients_quantity,percent_of_return
total_income_category,,
A,25.0,92.00
B,5014.0,92.94
C,15921.0,91.50
D,349.0,93.98
E,22.0,90.91


### Влияние уровня дохода на возврат кредита в срок
**Вывод:** Наименьший возврат по кредиту в срок у людей с доходом категории D (90,91%). Увеличение дохода не влияет на увеличение процента по возвратам кредита в срок. 

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [30]:
def purpose_influence (x):
    result = {
    'clients_quantity': x['purpose_category'].count(),
    'percent_of_return': (1 - x['debt'].mean()) * 100
    }
    return pd.Series(result).round(2)
data.groupby('purpose_category').apply(purpose_influence)

,clients_quantity,percent_of_return
purpose_category,,
операции с автомобилем,4279.0,90.65
операции с недвижимостью,10751.0,92.74
получение образования,3988.0,90.75
проведение свадьбы,2313.0,92.09


### Влияние цели получения кредита на его возврат в срок
**Вывод:** Наименьший возврат по кредитам в срок составляют кредиты на операции с автомобилями (90,65%) и получение образования (90,75%). Возврат в срок по кредитам на операции с недвижимостью (92,74%) и проведению свадеб (92,09%) находятся на примерно одинаковом уровне. 

### Шаг 4: общий вывод.

Основываясь на полученных данны можно сделать следующие выводы:
  1. Клиенты, узаконивающие свои отношения в органах гос. регистрации, являются более ответственными и платят по кредиту в срок в среднем на 2% чаще. 
  2. Наличие детей в семье играет скорее отрицательный эффект на возвращаемость кредита в срок, так как адекватно оценить влияние 5 детей в семьей невозможно, по причине недостаточной выборки (0,04%). Клиенты без детей на 1,5-2% чаще возвращают кредит в срок. 
  3. Увеличение дохода клиента не влияет на увеличение возвращаемости. Самый низкий процент возвращаемости у людей с очень низким доходом E (90,91%), самый высокий у клиентов с низким доходом D (93.98). 
  4. Цель кредита также оказывает влияние на возвращаемость кредита в срок. Клиенты которые берут кредит на операции с автомобилем (90,65%) и обучение (90,75%) реже платят в срок, чем клиенты, которые берут кредит на операции с недвижимостью (92,74%) и проведение свадьбы (92,09%)    